# HEART RATE ESTIMATION

## Introduction:

Seismocardiography(SCG) is a very promising technique to measure Heart Rate (HR) and Respiratory Rate (RR) with the detector positioned above sternum. It is generally based on accelerometer and gyroscope readings or a combination of them.

MuSe(Multi-Sensor miniaturized, low-power, wireless [IMU](https://en.wikipedia.org/wiki/Inertial_measurement_unit)) is an Inertial Measurement Unit (IMU) provide by [221e](https://www.221e.com). In the context of this project, It allows to record the inertial data necessary for the estimation of SCG.

The dataset used (**center_sternum.txt**) provide measurments acquired on a subject who was lying supine on his left and right side, respectively. MuSe  is placed on the center of the sternum

## Libraries:

The main libraries used to perform SCG are the following:

1. **Pandas**: to collect the dataset as DataFrame and select the columns of interest. 
* **Numpy**: to perform operations with arrays and use some usefull tools
* **Matplotlib**: to plot the resuls 
* **Seaborn**: to have a nice design of the plotted results 
* **Scipy**: to perform some numerical routines, such as linear algebra and statistics.

In [ ]:
######  LIBRARIES  #######
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
from scipy import linalg as la
from scipy import fft, ifft
from scipy.signal import welch, butter, filtfilt, find_peaks
import pywt
import seaborn as sns
import matplotlib


## Data preparation:

In this first part the dataset is loaded as DataFrame. The columns of interest are selected to do a complete data analysis. A "Time" column is added. This one will be usefull in the last part of the project to understand at which time a heart beat occurs. 

The selected data are then plotted. These are the three components of both the Accelerometer and Gyroscope. Thanks to these plots we are able to have an opening view of how these signals look like. Moreover a very noisily area at the begin and at the end of those signals can be detected. As consequence the signals are then cutted to select just the central part. These part correspond to exactly 60 seconds of data collections. 

1. Load the txt file and select only the columns you are interesting in, in order to do a complete data analysis (e.g. Log Freq, AccX, ... )

In [ ]:
###### IMPORT DATA #######
file_name = "center_sternum.txt"

# creation of a DataFrame containing the whole dataset
df_sternum = pd.DataFrame(pd.read_csv(file_name, sep = "\t", header = 0))

# select just the column of interest
df_sternum = df_sternum.iloc[:,[1,3,4,5,6,7,8]]

# recover the samples frequency
freq = df_sternum['Log Freq'][0]

# create the time vector
time = np.arange(0, len(df_sternum)* 1/freq, 1/freq)

# create the AccX, AccY, AccZ ,GyroX, GyroY, GyroZ vectors
accX, accY, accZ = df_sternum['AccX'][:], df_sternum['AccY'][:], df_sternum['AccZ'][:]
gyroX,gyroY,gyroZ = df_sternum['GyroX'][:], df_sternum['GyroY'][:], df_sternum['GyroZ'][:]

# insert time column
df_sternum.insert(1, "Time", time)

# print data
df_sternum.head()


2. Plot selected data in function of time and choose a properly time window over which to perform the analysis. Pay attention on time rappresentation and the measurament unit.

In [ ]:
##### PLOT OF THE DATA #####
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
sns.set()
# create the subplots 
figure, axis = plt.subplots(1,2, figsize = (15,5))

# plot the accelerometer's data
axis[0].plot(time,accX,'r',label = 'AccX')
axis[0].plot(time,accY,'b',label = 'AccY')
axis[0].plot(time,accZ,'g',label = 'AccZ')
axis[0].set_xlabel('Time [s]')
axis[0].set_ylabel('Acceleration [mg]')
axis[0].set_title('ACCELEROMETER')
axis[0].legend(loc = 1)

#plot the gyroscope's data
axis[1].plot(time,gyroX,'r',label = 'gyroX')
axis[1].plot(time,gyroY,'b',label = 'gyroY')
axis[1].plot(time,gyroZ,'g',label = 'gyroZ')
axis[1].set_xlabel('Time [s]')
axis[1].set_ylabel('Angular velocity [dps]')
axis[1].set_title('GYROSCOPE')
axis[1].legend(loc = 1)

# create the subplots for a detail of the previous ones
figure, axis = plt.subplots(1,2, figsize = (15,5))

# plot a detail of the accelerometer plot
axis[0].plot(time,accX,'r',label = 'AccX')
axis[0].plot(time,accY,'b',label = 'AccY')
axis[0].set_xlabel('Time [s]')
axis[0].set_ylabel('Acceleration [mg]')
axis[0].set_title('Detail of AccX and AccY')
axis[0].set_ylim([-80,150])
axis[0].set_xlim([10,25])
axis[0].legend(loc = 1)

# plot a detail of the gyroscope plot
axis[1].plot(time,gyroX,'r',label = 'gyroX')
axis[1].plot(time,gyroY,'b',label = 'gyroY')
axis[1].plot(time,gyroZ,'g',label = 'gyroZ')
axis[1].set_xlabel('Time [s]')
axis[1].set_ylabel('Angular velocity [dps]')
axis[1].set_title('GYROSCOPE')
axis[1].set_ylim([-10,10])
axis[1].set_xlim([10,25])
axis[1].legend(loc = 1)

plt.show()

In [ ]:
# select interval of interest to avoid the large amount of noise at the beginning and at the end of the original
# signal
df_sternum = df_sternum[(df_sternum['Time'] >= 10) & (df_sternum['Time'] < 70)]

# update accX, accY, accZ, gyroX, gyroY, gyroZ according to the new interval of interest
time = np.arange(0, len(df_sternum)* 1/freq, 1/freq)
accX, accY, accZ = df_sternum['AccX'][:], df_sternum['AccY'][:], df_sternum['AccZ'][:]
gyroX, gyroY, gyroZ = df_sternum['GyroX'][:], df_sternum['GyroY'][:], df_sternum['GyroZ'][:]


3. In order to make an appropiate work, decide if take care about some particular axis or some combination of them as well as derived features for the next step of the task. Motivate your choice.

**PCA** (Principal Component Analysis) is a dimensionality reduction technique. It's used to extract relevant information in a big confusing dataset. The goal of PCA is to find a new basis to re-express a dataset in order to reveal interesting stucture.

The central idea of PCA is to find out redudancy between variables and remove it as much as possibile. The main steps of PCA are the following: 
1. Compute the data mean vector from the data matrix X. 
* Subtract off mean vector from the dataset.
* Calculate the sample covariance matrix Cx.
* Calculate eigenvectors of matrix Cx in order to obtain the matrix P (new base).
* Apply the change of base PX = Y. Y will be the tranformed data matrix, if digonal, is uncurrelated and so we won't have any redundacy.


**PCA for Accelerometer**

In this section the PCA is used over the three components of the Accelerometer. 

As consequence of the results, a new base is apply to the data matrix and we reduce it from three to two dimensions. This operation allow us to keep and amount of 91% of information from the original matrix.

In [ ]:
# create the matrix for the PCA technique, in the rows there are the AccX, AccY, AccZ measurments, and each 
# column represents a data sample

matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
sns.set()
X_acc = np.array([accX, accY, accZ])

figure, axis = plt.subplots(1,3, figsize = (18,5))
axis[0].scatter(X_acc[0,:], X_acc[1,:], alpha=0.3, color = 'b')
axis[0].set_xlabel('AccX')
axis[0].set_ylabel('AccY')
axis[1].scatter(X_acc[0,:], X_acc[2,:], alpha=0.3, color = 'b')
axis[1].set_xlabel('AccX')
axis[1].set_ylabel('AccZ')
axis[2].scatter(X_acc[1,:], X_acc[2,:], alpha=0.3, color = 'b')
axis[2].set_xlabel('AccY')
axis[2].set_ylabel('AccZ')
plt.show()

In [ ]:
# compute the mean of X and subtract it to obtain a zero main matrix X
X_acc = X_acc - X_acc.mean()

# calculate the eigenvalues and eigenvectors of the covariance matrix and compute the covariance matrix of the new 
# matrix 
l, V = la.eig(np.cov(X_acc))
#P = V.T
#Cy = P.dot(np.cov(X_acc)).dot(P.T)
Cy = np.diag(np.round(np.real(l),2))
print(Cy, "\n")


# estimate if reduce dimensionality is convenient 
print("Trace of the original data matrix: " , np.round(np.cov(X_acc).trace(),2))
print("Trace of the data matrix after the changes of base: ", np.round(Cy.trace(),2), "\n")

print("Fraction of the total variability keeping only the first principal component: ", np.round(100*(Cy[0,0])/Cy.trace()),"%")
print("Fraction of the total variability keeping only the first two principal components: ", np.round(100*(Cy[0,0]+Cy[1,1])/Cy.trace()),"%")
print("Fraction of the total variability keeping only the last two principal components: ", np.round(100*(Cy[1,1]+Cy[2,2])/Cy.trace()),"%")

# as consequence of the PCA technique keeping the two principal components for the Accelerometer is better

P = V[:,0:2] #the eigenvectors are in the columns of V

PCA_acc = P.T.dot(X_acc) # new data according to PCA result

#Visualize the original data projected into the new space with the relative principal axis
scale_factor = 0.001
first_dim = PCA_acc[0] - PCA_acc[0].mean()
second_dim = PCA_acc[1] - PCA_acc[1].mean()
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
sns.set()
plt.figure(figsize=(10,6))
plt.scatter(first_dim, second_dim, alpha=0.3, c = 'b')
plt.plot([0, scale_factor*l[0]*np.cov(PCA_acc)[0,0]], [0, scale_factor*l[0]*np.cov(PCA_acc)[1,0]], 'r', lw=3)
plt.plot([0, scale_factor*l[1]*np.cov(PCA_acc)[0,1]], [0, scale_factor*l[1]*np.cov(PCA_acc)[1,1]], 'r', lw=3)
plt.title('PCA: ACCELEROMETER')
plt.xlabel('First dimension')
plt.ylabel('Second dimension')
plt.show()

**PCA for Gyroscope**

In this section the PCA is used over the three components of the Gyroscope. 

As consequence of the results, a new base is apply to the data matrix and we reduce it from three to two dimensions. This operation allow us to keep and amount of 96% of information from the original matrix.

In [ ]:
X_gyro = np.array([gyroX, gyroY, gyroZ]) 
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
sns.set()
figure, axis = plt.subplots(1,3, figsize = (18,5))
axis[0].scatter(X_gyro[0,:], X_gyro[1,:], alpha=0.3, color = 'b')
axis[0].set_xlabel('GyroX')
axis[0].set_ylabel('GyroY')
axis[1].scatter(X_gyro[0,:], X_gyro[2,:], alpha=0.3, color = 'b')
axis[1].set_xlabel('GyroX')
axis[1].set_ylabel('GyroZ')
axis[2].scatter(X_gyro[1,:], X_gyro[2,:], alpha=0.3, color = 'b')
axis[2].set_xlabel('GyroY')
axis[2].set_ylabel('GyroZ')
plt.show()

In [ ]:
#compute the mean of X and subtract it to obtain a zero main matrix X
X_gyro = X_gyro - X_gyro.mean()

# calculate the eigenvalues and eigenvectors of the covariance matrix and compute the covarinace matrix of the new 
# matrix 
l, V = la.eig(np.cov(X_gyro))
#P = V.T
#Cy = P.dot(np.cov(X)).dot(P.T)
Cy = np.diag(np.round(np.real(l),2))
print(Cy, "\n")


# estimate if reduce dimensionality is convenient 
print("Trace of the original data matrix: " , np.round(np.cov(X_gyro).trace(),2))
print("Trace of the data matrix after the changes of base: ", np.round(Cy.trace(),2), "\n")

print("Fraction of the total variability keeping only the first principal component: ", np.round(100*(Cy[0,0])/Cy.trace()),"%")
print("Fraction of the total variability keeping only the first two principal components: ", np.round(100*(Cy[0,0]+Cy[1,1])/Cy.trace()),"%")
print("Fraction of the total variability keeping only the last two principal components: ", np.round(100*(Cy[1,1]+Cy[2,2])/Cy.trace()),"%")

# as consequence of the PCA technique keeping the two principal components for the Gyroscope is better
P = V[:,0:2] #the eigenvectors are in the columns of V

PCA_gyro = P.T.dot(X_gyro) # new data according to PCA result


#Visualize the original data projected into the new space with the relative principal axis
first_dim = PCA_gyro[0] - PCA_gyro[0].mean()
second_dim = PCA_gyro[1] - PCA_gyro[1].mean()
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
sns.set()
plt.figure(figsize=(10,6))
plt.scatter(first_dim, second_dim, alpha=0.3, c = 'b')
plt.plot([0, l[0]*np.cov(PCA_gyro)[0,0]], [0, l[0]*np.cov(PCA_gyro)[1,0]], 'r', lw=2)
plt.plot([0, l[1]*np.cov(PCA_gyro)[0,1]], [0, l[1]*np.cov(PCA_gyro)[1,1]], 'r', lw=2)
plt.title('PCA: GYROSCOPE')
plt.xlabel('First dimension')
plt.ylabel('Second dimension')
plt.show()

## Time and frequency analysis:

1. Statistical analysis: provide a statistical description of the chosen dataset. Statistical descriptors includes for example mean, median, variance, standard deviation, 25th and 75th percentiles, and correlation coefficients. Investigate what could be the most interesting descriptors for this type of data, motivating the choices.


In [ ]:
# Create dataframe with the component obtained from PCA

A = [PCA_acc[0].reshape(len(PCA_acc[0]), 1), PCA_acc[1].reshape(len(PCA_acc[1]), 1), PCA_gyro[0].reshape(len(PCA_gyro[0]), 1), PCA_gyro[1].reshape(len(PCA_gyro[1]), 1)]
A = np.array(A).reshape(len(PCA_acc[0]), 4)
df_PCA = pd.DataFrame(data = A)
df_PCA.insert(0, "Time", time)

# compute statistics
df_PCA = df_PCA.describe()
df_PCA = df_PCA.rename(columns={"Time": "Time [s]", 0 : "AccX [mg]", 1: "AccY [mg]", 2: "GyroX [rad/s]", 3: "GyroY [rad/s]"})
df_PCA.round(1)
stat = df_PCA.iloc[:,1:].round(1) 

# TODO: add correlation coeffs grassie
stat



2. Fourier Analysis: Perform a frequency analysis of the data. Look at the spectrum and explain what you see. Use this step in order to properly design the filters in the following step. 


In [ ]:
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)      
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])     
    return f_values, fft_values

def get_psd_values(y_values, T, N, f_s):
    f_values, psd_values = welch(y_values, fs=f_s)
    return f_values, psd_values

N = df_sternum.shape[0]
T = 1/freq   
f_s = freq   

f_accX, fft_accX = get_fft_values(np.array(PCA_acc[0]), T, N, f_s)
f_accY, fft_accY  = get_fft_values(np.array(PCA_acc[1]), T, N, f_s)
f_gyroX, fft_gyroX = get_fft_values(np.array(PCA_gyro[0]), T, N, f_s)
f_gyroY, fft_gyroY = get_fft_values(np.array(PCA_gyro[1]), T, N, f_s)

figure, axis = plt.subplots(2, 2, figsize = (15, 5))
axis[0, 0].plot(f_accX, fft_accX)
axis[0, 0].set_ylim([0, 2])
axis[0, 1].plot(f_accY, fft_accY)
axis[0, 1].set_ylim([0, 2])
axis[1, 0].plot(f_gyroX, fft_gyroX)
axis[1, 0].set_ylim([0, 2])
axis[1, 1].plot(f_gyroY, fft_gyroY)
axis[1, 1].set_ylim([0, 2])

plt.show()


In [ ]:
f_accX_psd, fft_accX_psd = get_psd_values(np.array(PCA_acc[0]), T, N, f_s)
f_accY_psd, fft_accY_psd  = get_psd_values(np.array(PCA_acc[1]), T, N, f_s)
f_gyroX_psd, fft_gyroX_psd = get_psd_values(np.array(PCA_gyro[0]), T, N, f_s)
f_gyroY_psd, fft_gyroY_psd = get_psd_values(np.array(PCA_gyro[1]), T, N, f_s)

figure, axis = plt.subplots(2, 2, figsize = (15, 5))
axis[0, 0].plot(f_accX_psd, fft_accX_psd)
#axis[0, 0].set_ylim([0, 2])
axis[0, 1].plot(f_accY_psd, fft_accY_psd)
#axis[0, 1].set_ylim([0, 2])
axis[1, 0].plot(f_gyroX_psd, fft_gyroX_psd)
#axis[1, 0].set_ylim([0, 2])
axis[1, 1].plot(f_gyroY_psd, fft_gyroY_psd)
#axis[1, 1].set_ylim([0, 2])

plt.show()

**Filter:**
    
Implement your own filter, trying to extrapolate heart rate signal. Hint:
    
    (a) Directly from Fourier Analysis, antitrasform data looking for the most interesting frequency band.
    
    (b) Choose the appropriate Lowpass/Bandpass/Highpass filter.
    
    (c) Wavelet trasform (a powerfull instrument that make a time and frequency analysis of signal).
    
    (d) Find another method by yourselves.
    
Motivate your choice.
    

In [ ]:
# Normalize
selected = (PCA_gyro[0]-np.mean(PCA_gyro[0]))
fs = 200
lf = 40/60 #bmp to Hz
hf = 120/60 

In [ ]:
def fftfilt(x,lf,hf,fs):
    X = np.fft.rfft(x)
    f = np.fft.rfftfreq(len(x),1/fs)
    X_filt = np.array([X[i] if (f[i] >= lf and f[i] <= hf) else 0 for i in range(0,len(X))])
    x_filt = np.fft.irfft(X_filt)
    return x_filt

def butterfilt(x, lf, hf, fs):
    b,a = butter(4,[lf,hf], btype='bandpass', fs=fs)
    x_filt = filtfilt(b,a, x)  
    return x_filt 

def wavelet(x):
    # 1. Decompose the signal using the DWT.
    # 2. Filter the signal in the wavelet space using thresholding.
    # 3. Invert the filtered signal to reconstruct the original, now filtered signal, using the inverse DWT.
    
    # cut frequencies 
    
    # DWT
    cA, cD = pywt.dwt(x,'sym4') 

    # Compute threshold something like this.  You need an estimate of the noise sigma.
    threshold=1*math.sqrt(2*math.log2(x.size))

    # Apply the threshold.
    cA_threshold = pywt.threshold(cA, threshold, 'soft')
    cD_threshold = pywt.threshold(cD, threshold, 'soft')
    
    # IDWT
    renc = pywt.idwt(cA_threshold, cD_threshold, 'sym4'); 
    
    return renc


In [ ]:
# Computation
sig_filt_fft = fftfilt(selected,lf,hf,fs)
sig_filt_butter = butterfilt(selected,lf,hf,fs)
sig_filt_wavelet = wavelet(selected)

# Plot
figure, axis = plt.subplots(3, 1, figsize = (20, 15))

axis[0].plot(time, selected, 'green')
axis[0].plot(time, sig_filt_fft, 'red')
axis[0].set_xlim([30,40])

axis[1].plot(time, selected, 'green')
axis[1].plot(time, sig_filt_butter, 'red')
axis[1].set_xlim([30,40])

axis[2].plot(time, sig_filt_wavelet, 'red')
axis[2].plot(time, selected, 'green')
axis[2].set_xlim([30,40])

plt.show()


In [ ]:
# Show choosen filter
figure, axis = plt.subplots(3, 1, figsize = (20, 15))

axis[0].plot(time, sig_filt_wavelet, 'black')
axis[0].set_xlim([0,1])

axis[1].plot(time, sig_filt_wavelet, 'black')
axis[1].set_xlim([32,36])

axis[2].plot(time, sig_filt_wavelet, 'black')
axis[2].set_xlim([0,60])

plt.show()

**Metrics:**

1. Heart Beat Per Minute(BPM): extrapolate BPM, make an histogram of the result. Does it follow a partiular distribution? 
    

In [ ]:
# Create histogram 
# FIND ALL PEAKS ASSOCIATI AL SECONDO, PER TROVARE IL PASSO 

peak_times = []
peak_time = 0

for i in range(1, len(sig_filt_wavelet)):     
    if abs(sig_filt_wavelet[i]) > 0.:
        peak_time = i/200
        peak_time = round(peak_time % 1, 1)
        peak_times.append(peak_time)
    
print(peak_times)
    

In [ ]:
import statistics

print(statistics.mean(peak_times)) #BPM = 60 / mean

In [ ]:
#fare hist e gaussiana calcolare mean e variance
# BPM = mean +- varianze
#PER MR TAMBURINO

In [ ]:
#cerco i picchi (battiti) 
peaks_fft,_ = find_peaks(sig_filt_fft, height=0.2, threshold=None, distance=fs/hf, prominence=0.4, width=None, wlen=None, rel_height=0.5, plateau_size=None)
peaks_butt,_ = find_peaks(sig_filt_butter, height=0.2, threshold=None, distance=fs/hf, prominence=0.4, width=None, wlen=None, rel_height=0.5, plateau_size=None)
peaks_wavelet,_ = find_peaks(sig_filt_wavelet, height=0.2, threshold=None, distance=fs/hf, prominence=0.4, width=None, wlen=None, rel_height=0.5, plateau_size=None)
print("mean BPM with FFT filt= ",len(peaks_fft)*60/(len(sig_filt_fft)/fs))
print("mean BPM with butterworth filt= ",len(peaks_butt)*60/(len(sig_filt_butter)/fs))
print("mean BPM with wavelet filt= ",len(peaks_wavelet)*60/(len(sig_filt_wavelet)/fs))


In [ ]:
array = np.full(len(peaks_wavelet), 2)

plt.figure(figsize=(20, 10))
plt.plot(time, sig_filt_wavelet, 'black')
plt.plot(peaks_wavelet/200, array, 'o')
plt.xlim(9,20)

# TODO: DA FARE

2. Heart Rate Variability(HRV): extrapolate HRV, explain why this parameter is important, and plot the results.

**Optional Algorithm:** 

Elaborate a simple algorithm to extrapolate heart beat even when filter failed (e.g. look at particular threshold...).  

**Conclusion:**

Summarise the obtained results, in particular making a comparison between the two files analysed. Highlight limitation and critical issues encountered during the work, motivating the most relevant contribution given by your solution.

